## 🧪 White-box Testing: Branch Coverage under Different Python Versions

### 1️⃣ Background and Objectives
> This test verifies whether the `pickle` module produces identical serialized outputs under different Python versions (3.7.12 vs. 3.12.4) on the same OS, using white-box branch coverage techniques.


### 2️⃣ Environment Information
> We record the operating system and Python version in use.

In [1]:
import platform
import sys

def print_environment_info():
    print("📌 Current operating system:", platform.system(), platform.release())
    print("📌 Python version:", sys.version)

In [2]:
print_environment_info()

📌 Current operating system: Windows 10
📌 Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 05:35:01) [MSC v.1916 64 bit (AMD64)]


In [2]:
print_environment_info()

📌 Current operating system: Windows 11
📌 Python version: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]


### 3️⃣ Test Cases and Input Structures
> Each input structure is mapped to a specific branch logic (e.g., empty vs. non-empty lists).

In [3]:
import pickle
import hashlib

def get_hash(obj):
    """Return the SHA256 hash value of the object after pickle serialization"""
    return hashlib.sha256(pickle.dumps(obj)).hexdigest()

In [4]:
import os

def save_result_by_python_version(case_name, value_hash, file_path="branch_python_hashes.txt"):
    py_version = ".".join(sys.version.split(" ")[0].split(".")[:2])
    block = f"{case_name}\n{py_version.ljust(10)}Result: {value_hash}\n"

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
    else:
        content = ""

    blocks = content.strip().split("\n\n") if content else []
    updated = False

    for i in range(len(blocks)):
        if blocks[i].startswith(case_name):
            lines = blocks[i].split("\n")
            lines = [line for line in lines if not line.startswith(py_version)]
            lines.append(f"{py_version.ljust(10)}Result: {value_hash}")
            blocks[i] = "\n".join(lines)
            updated = True
            break

    if not updated:
        blocks.append(block.strip())

    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n\n".join(blocks) + "\n")


In [5]:
def run_branch_tests_by_version():
    save_result_by_python_version("TC_BC_01 Empty list", get_hash([]))
    save_result_by_python_version("TC_BC_02 Non-empty list", get_hash([1]))
    save_result_by_python_version("TC_BC_03 Empty dict", get_hash({}))
    save_result_by_python_version("TC_BC_04-A Dict a→b", get_hash({"a": 1, "b": 2}))
    save_result_by_python_version("TC_BC_04-B Dict b→a", get_hash({"b": 2, "a": 1}))
    save_result_by_python_version("TC_BC_05-A Nested x→y", get_hash({"outer": {"x": 1, "y": 2}}))
    save_result_by_python_version("TC_BC_05-B Nested y→x", get_hash({"outer": {"y": 2, "x": 1}}))
    
class MyClass:
    def __reduce__(self):
        return (MyClass, ())
        
save_result_by_python_version("TC_BC_06 Custom class", get_hash(MyClass()))
save_result_by_python_version("TC_BC_07 Float list", get_hash([0.1, 0.2, 0.3]))

In [6]:
run_branch_tests_by_version()

In [7]:
def print_version_results(file_path="branch_python_hashes.txt"):
    if not os.path.exists(file_path):
        print("❌ No result file found.")
        return
    with open(file_path, "r", encoding="utf-8") as f:
        print("✅ Recorded results:\n")
        print(f.read())

### 4️⃣ Platform-specific Hash Results
> Hash outputs for each test case executed on macOS, Windows, and Linux.

In [8]:
print_version_results("branch_python_hashes.txt")

✅ Recorded results:

TC_BC_01 Empty list
3.12      Result: ec0a6ccf9debf1c16781445c4b9106080d00478b0559469336db7c7b7b9711c8
3.7       Result: 08e14d71fb56a8d6888e9b1f0c21d1a98050e7cdf034fdfcd06d0db59f810477

TC_BC_02 Non-empty list
3.12      Result: 4161bccd029bec6dfae5cc2e6620fc7864b70ff481c3a1117f9926854a9b1974
3.7       Result: d28c71251ff6125a6b6d3d2f5a6881d5f73986307ea6439dbced77c5a471b604

TC_BC_03 Empty dict
3.12      Result: 926248e52d1fa532c317e37da24ed652ae64110f8219cb5e061668bd3091f048
3.7       Result: 8ec16a78d9c9d3646c559df239aea3f15333605ddc702eeacebf6dd3ebeff684

TC_BC_04-A Dict a→b
3.12      Result: dfc78ac090a4eb6e9db8fc6475a84cd700c8cc0e389778312ad491d662e081e1
3.7       Result: 5ee253f93ea5b78548bb8e7e8d8eaee3ee365076c2e1cceca0a4878f52a4b1a6

TC_BC_04-B Dict b→a
3.12      Result: ae8622c40986e49d221286df26c06338a0d074dbe60d86b8f52cb52cf585cf1a
3.7       Result: 0439f5da70849ad2417e47acd7bac2e0e877049bc7c5348857c13cf1462cf3d3

TC_BC_05-A Nested x→y
3.12      Result: 

### 5️⃣ Consistency Analysis and Divergence Detection

All test cases were executed on the **same operating system** using two different Python versions: **3.12.4** and **3.7.12**. The outputs of `pickle.dumps()` were hashed using SHA256 and compared between the two versions.

**Result:** For all test cases, the resulting hashes **differed** between Python versions.

#### ⚠️ Test cases with inconsistent hashes between Python 3.12 and Python 3.7:

| Test Case ID     | Description                            |
|------------------|----------------------------------------|
| TC_BC_01         | Empty list                             |
| TC_BC_02         | Non-empty list                         |
| TC_BC_03         | Empty dictionary                       |
| TC_BC_04-A       | Dict with keys ordered a → b           |
| TC_BC_04-B       | Dict with keys ordered b → a           |
| TC_BC_05-A       | Nested dict with keys x → y            |
| TC_BC_05-B       | Nested dict with keys y → x            |
| TC_BC_06         | Custom class object with `__reduce__`  |
| TC_BC_07         | Floating-point list                    |

<br>

> Even though the inputs were exactly the same and semantically equivalent, all test cases produced different hash values across Python versions. This confirms that the internal representation and serialization logic of `pickle` has changed between Python 3.7 and 3.12.


### 6️⃣ Conclusions and Findings

When tested on the **same operating system**, the `pickle` module produced **non-deterministic outputs** across Python 3.12.4 and Python 3.7.12. Despite using identical inputs, all test cases resulted in different SHA256 hashes between the two versions.

#### 🔍 Key Findings:

- ❌ All test cases produced different hash results between Python 3.12 and Python 3.7;
- ⚙️ The differences are likely due to internal implementation changes in object serialization across versions;
- ❗ Even simple and semantically identical data structures (e.g., `[]`, `{}`, custom classes) yielded diverging outputs.

#### ⚠️ Limitations:

- Only two versions of Python were tested: 3.12.4 and 3.7.12;
- The pickle protocol version was not explicitly varied (defaults used);
- The test suite did not cover advanced or nested object models (e.g., circular references, shared references);
- Only one operating system was used; cross-system + cross-version interaction was not assessed.

> Future work should include broader Python version coverage, control over pickle protocol levels, and fuzzed inputs to investigate compatibility boundaries and serialization determinism in more depth.


### 📎 Appendix: Raw Data File

The complete platform hash records can be found in the following file:

👉 [Download branch_python_hashes.txt](./branch_python_hashes.txt)